In [6]:
import pandas as pd 
from datetime import datetime, timedelta
from sqlalchemy import create_engine, text

In [7]:
# Create an engine that stores data in the local directory's
engine = create_engine('sqlite:///app.db', echo=True)

campaigns target 

campaigns shared_url 

campaigns_profile 

campaign code : code promo 

In [8]:
create_campaigns_table = text('''CREATE TABLE IF NOT EXISTS campaigns (
    campaign_id INTEGER PRIMARY KEY AUTOINCREMENT,
    company_id INTEGER NOT NULL,                       
    title VARCHAR(255) NOT NULL,
    description TEXT,
    start_date DATE,
    end_date DATE,
    campaign_status VARCHAR(100),
    campaign_code VARCHAR(100),
    target_audience VARCHAR(255),
    budget DECIMAL(10, 2),
    actual_spending DECIMAL(10, 2),
    campaign_result TEXT,
    FOREIGN KEY (company_id) REFERENCES companies(company_id) ON DELETE CASCADE
);''')


In [12]:
# Assuming 'engine' is already defined and imported from SQLAlchemy setup
with engine.connect() as connection:
    connection.execute(create_campaigns_table)
    print("Campaigns table created successfully.")

2024-04-18 11:53:53,115 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-18 11:53:53,116 INFO sqlalchemy.engine.Engine CREATE TABLE IF NOT EXISTS campaigns (
    campaign_id INTEGER PRIMARY KEY AUTOINCREMENT,
    company_id INTEGER NOT NULL,                       
    title VARCHAR(255) NOT NULL,
    description TEXT,
    start_date DATE,
    end_date DATE,
    campaign_status VARCHAR(100),
    campaign_code VARCHAR(100),
    target_audience VARCHAR(255),
    budget DECIMAL(10, 2),
    actual_spending DECIMAL(10, 2),
    campaign_result TEXT,
    FOREIGN KEY (company_id) REFERENCES companies(company_id) ON DELETE CASCADE
);
2024-04-18 11:53:53,118 INFO sqlalchemy.engine.Engine [cached since 10.45s ago] ()
Campaigns table created successfully.
2024-04-18 11:53:53,148 INFO sqlalchemy.engine.Engine ROLLBACK


In [10]:
# Assuming 'engine' is already defined and imported from SQLAlchemy setup
with engine.connect() as connection:
    connection.execute(text('''DROP TABLE IF EXISTS campaigns'''))

2024-04-18 11:53:46,616 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-18 11:53:46,617 INFO sqlalchemy.engine.Engine DROP TABLE IF EXISTS campaigns
2024-04-18 11:53:46,618 INFO sqlalchemy.engine.Engine [generated in 0.00126s] ()
2024-04-18 11:53:46,624 INFO sqlalchemy.engine.Engine ROLLBACK


In [13]:
pd.read_sql('SELECT * FROM campaigns',engine)

2024-04-18 11:53:56,125 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-18 11:53:56,129 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM campaigns")
2024-04-18 11:53:56,129 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-18 11:53:56,131 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM campaigns")
2024-04-18 11:53:56,132 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-18 11:53:56,133 INFO sqlalchemy.engine.Engine SELECT * FROM campaigns
2024-04-18 11:53:56,133 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-18 11:53:56,136 INFO sqlalchemy.engine.Engine ROLLBACK


,campaign_id,company_id,title,description,start_date,end_date,campaign_status,campaign_code,target_audience,budget,actual_spending,campaign_result


In [16]:
def add_campaign(company_id,title,**kwargs):
     data = {'company_id': company_id,'title':title,**kwargs}
     columns = ', '.join(data.keys())
     placeholders = ', '.join(f":{key}" for key in data.keys())

     insert_query = text(f"""
            INSERT INTO campaigns ({columns})
            VALUES ({placeholders});
        """)
     with engine.begin() as connection:
        connection.execute(insert_query, data)
        print(f"Campaign added")

def modify_campaign(campaign_id,**kwargs):
    updates = ', '.join(f"{key} = :{key}" for key in kwargs)
    update_query = text(f"""
        UPDATE campaigns
        SET {updates}
        WHERE campaign_id = :campaign_id;
    """)
    kwargs['campaign_id'] = campaign_id  # Include for WHERE clause
    with engine.begin() as connection:
        result = connection.execute(update_query, kwargs)
        if result.rowcount > 0:
            print("Campaign updated")
        else:
            print(f"No Campaign found with ID.")

def get_follower(**kwargs):
    if not kwargs:
        print("No parameters provided to search for campaign.")
        return None

    # Constructing the WHERE clause based on kwargs
    conditions = [f"{key} = :{key}" for key in kwargs]
    where_clause = " AND ".join(conditions)
    select_query = text(f"SELECT * FROM campaigns WHERE {where_clause}")

    print(select_query)  # Optional: Print the query for debugging
    with engine.connect() as connection:
        result = connection.execute(select_query, kwargs).fetchone()
        print(result)  # Optional: Print the raw result for debugging
        
        if result:
            # Assuming we know the columns in order of the table
            campaign_dict = {
                'campaign_id': result[0],
                'company_id': result[1],
                'title': result[2],
                'description': result[3],
                'start_date': result[4],
                'end_date': result[5],
                'campaign_status': result[6],
                'campaign_code': result[7],
                'target_audience': result[8],
                'budget': result[9],
                'actual_spending': result[10],
                'campaign_result': result[11],
            }
            
            return campaign_dict
        else:
            print("Campaign not found.")
            return None
        
def delete_campaign(campaign_id):
    delete_query = text("DELETE FROM companies WHERE campaign_id = :campaign_id")

    with engine.begin() as connection:
        result = connection.execute(delete_query, {'campaign_id': campaign_id})
        if result.rowcount > 0:
            print(f"Campaign deleted successfully.")
        else:
            print(f"No campaign found nothing deleted.")


In [20]:
add_campaign(company_id = 1,title='')

2024-04-18 11:56:05,427 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-18 11:56:05,428 INFO sqlalchemy.engine.Engine 
            INSERT INTO campaigns (company_id, title)
            VALUES (?, ?);
        
2024-04-18 11:56:05,433 INFO sqlalchemy.engine.Engine [cached since 25.88s ago] (1, '')
Campaign added
2024-04-18 11:56:05,492 INFO sqlalchemy.engine.Engine COMMIT


In [21]:
pd.read_sql('SELECT * FROM campaigns', engine)

2024-04-18 11:56:07,732 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-18 11:56:07,733 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM campaigns")
2024-04-18 11:56:07,734 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-18 11:56:07,735 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM campaigns")
2024-04-18 11:56:07,737 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-18 11:56:07,740 INFO sqlalchemy.engine.Engine SELECT * FROM campaigns
2024-04-18 11:56:07,740 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-18 11:56:07,743 INFO sqlalchemy.engine.Engine ROLLBACK


,campaign_id,company_id,title,description,start_date,end_date,campaign_status,campaign_code,target_audience,budget,actual_spending,campaign_result
0,1,1,dede,None,None,None,None,None,None,None,None,None
1,2,1,,None,None,None,None,None,None,None,None,None


In [22]:
modify_campaign(campaign_id=2 ,title='Le nouveau titre hehehe')

2024-04-18 11:57:10,662 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-18 11:57:10,663 INFO sqlalchemy.engine.Engine 
        UPDATE campaigns
        SET title = ?
        WHERE campaign_id = ?;
    
2024-04-18 11:57:10,663 INFO sqlalchemy.engine.Engine [generated in 0.00078s] ('Le nouveau titre hehehe', 2)
Campaign updated
2024-04-18 11:57:10,688 INFO sqlalchemy.engine.Engine COMMIT


In [23]:
pd.read_sql('SELECT * FROM campaigns', engine)

2024-04-18 11:57:12,980 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-18 11:57:12,981 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM campaigns")
2024-04-18 11:57:12,981 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-18 11:57:12,982 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM campaigns")
2024-04-18 11:57:12,982 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-18 11:57:12,982 INFO sqlalchemy.engine.Engine SELECT * FROM campaigns
2024-04-18 11:57:12,982 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-18 11:57:12,988 INFO sqlalchemy.engine.Engine ROLLBACK


,campaign_id,company_id,title,description,start_date,end_date,campaign_status,campaign_code,target_audience,budget,actual_spending,campaign_result
0,1,1,dede,None,None,None,None,None,None,None,None,None
1,2,1,Le nouveau titre hehehe,None,None,None,None,None,None,None,None,None
